first cell is dataset generation and 2nd one is formula

In [ ]:
import numpy as np
import pandas as pd
from math import radians, sin, cos, sqrt, atan2, log10, exp, pi
from dataclasses import dataclass
from typing import Tuple, Dict, List, Optional, Union
from datetime import datetime, timedelta
import os
from scipy.spatial.distance import cdist
from scipy.special import j0  # Bessel function for Jake's model

# ----------------------------
# Enhanced utilities & constants
# ----------------------------
EARTH_R = 6371000.0  # meters
C = 299792458.0       # m/s

def haversine_m(lat1, lon1, lat2, lon2):
    """Great-circle distance in meters."""
    p1, p2 = radians(lat1), radians(lat2)
    dphi = radians(lat2 - lat1)
    dlambda = radians(lon2 - lon1)
    a = sin(dphi/2)*2 + cos(p1)*cos(p2)*sin(dlambda/2)*2
    return EARTH_R * 2 * atan2(sqrt(a), sqrt(1 - a))

def db_to_lin(db):
    return 10 ** (db / 10.0)

def lin_to_db(lin):
    return 10 * np.log10(np.maximum(lin, 1e-30))

def clamp(val, vmin, vmax):
    return max(vmin, min(vmax, val))

def calculate_angle_deg(lat1, lon1, lat2, lon2):
    """Calculate bearing angle from point 1 to point 2 in degrees."""
    dlon = radians(lon2 - lon1)
    lat1, lat2 = radians(lat1), radians(lat2)
    y = sin(dlon) * cos(lat2)
    x = cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)
    bearing = atan2(y, x)
    return (bearing * 180 / pi + 360) % 360

# ----------------------------
# Enhanced Base Station class with multi-BS support
# ----------------------------
@dataclass
class BaseStation:
    lat: float
    lon: float
    power_dbm: float = 30.0
    height_m: float = 30.0
    antenna_gain_dbi: float = 15.0
    azimuth_deg: float = 0.0  # Main beam direction
    beamwidth_deg: float = 120.0  # 3dB beamwidth
    sector_id: int = 0
    frequency_hz: float = 2.4e9

    def antenna_gain_pattern(self, angle_deg: float) -> float:
        """Simple directional antenna pattern (3GPP-like)"""
        angle_diff = abs((angle_deg - self.azimuth_deg + 180) % 360 - 180)
        if angle_diff <= self.beamwidth_deg / 2:
            return self.antenna_gain_dbi
        else:
            # Side lobe attenuation
            return self.antenna_gain_dbi - 20  # 20 dB front-to-back ratio

# ----------------------------
# Enhanced Mobility Models
# ----------------------------
@dataclass
class Bounds:
    lat_min: float; lat_max: float; lon_min: float; lon_max: float

# 1. Real GPS trace mobility - ADD HERE
class TraceDrivenMobility:
    """Mobility model based on real GPS traces"""
    def _init_(self, trace_file: str, dt: float):
        """
        trace_file: CSV with columns ['timestamp', 'lat', 'lon', 'speed_mps']
        """
        self.trace_data = pd.read_csv(trace_file)
        self.dt = dt
        self.current_index = 0
        self.lat = self.trace_data.iloc[0]['lat']
        self.lon = self.trace_data.iloc[0]['lon']
        self.speed = self.trace_data.iloc[0]['speed_mps'] if 'speed_mps' in self.trace_data.columns else 0
        
    def step(self):
        if self.current_index < len(self.trace_data) - 1:
            self.current_index += 1
            row = self.trace_data.iloc[self.current_index]
            self.lat = row['lat']
            self.lon = row['lon'] 
            self.speed = row['speed_mps'] if 'speed_mps' in self.trace_data.columns else 0
        return self.lat, self.lon, self.speed

class GroupMobility:
    """Group mobility for modeling pedestrian groups, vehicle platoons"""
    def _init_(self, bounds: Bounds, dt: float, group_size: int = 5, cohesion_strength: float = 0.8):
        self.bounds = bounds
        self.dt = dt
        self.group_size = group_size
        self.cohesion = cohesion_strength
        
        # Initialize group center
        self.center_lat = np.random.uniform(bounds.lat_min, bounds.lat_max)
        self.center_lon = np.random.uniform(bounds.lon_min, bounds.lon_max)
        self.group_speed = np.random.uniform(1, 15)  # m/s
        self.group_direction = np.random.uniform(0, 2*pi)
        
        # Initialize individual positions around center
        self.members = []
        for i in range(group_size):
            offset_m = np.random.uniform(5, 20)  # 5-20m from center
            offset_angle = np.random.uniform(0, 2*pi)
            
            lat_offset = (offset_m * cos(offset_angle)) / 111000
            lon_offset = (offset_m * sin(offset_angle)) / (111000 * cos(radians(self.center_lat)))
            
            self.members.append({
                'lat': self.center_lat + lat_offset,
                'lon': self.center_lon + lon_offset,
                'individual_direction': np.random.uniform(0, 2*pi)
            })
    
    def step(self, member_id: int = 0):
        # Update group center
        m_per_deg_lat = 111000.0
        m_per_deg_lon = 111000.0 * cos(radians(self.center_lat))
        
        dx_m = self.group_speed * self.dt * cos(self.group_direction)
        dy_m = self.group_speed * self.dt * sin(self.group_direction)
        
        self.center_lat += dy_m / m_per_deg_lat
        self.center_lon += dx_m / m_per_deg_lon
        
        # Keep group center in bounds
        self.center_lat = clamp(self.center_lat, self.bounds.lat_min, self.bounds.lat_max)
        self.center_lon = clamp(self.center_lon, self.bounds.lon_min, self.bounds.lon_max)
        
        # Update individual member
        if member_id < len(self.members):
            member = self.members[member_id]
            
            # Cohesion force (attraction to group center)
            lat_to_center = self.center_lat - member['lat']
            lon_to_center = self.center_lon - member['lon']
            
            # Individual random movement
            individual_speed = self.group_speed * np.random.uniform(0.7, 1.3)
            
            # Combine group cohesion and individual movement
            new_direction = (self.cohesion * atan2(lon_to_center, lat_to_center) + 
                           (1 - self.cohesion) * member['individual_direction'])
            
            dx_m = individual_speed * self.dt * cos(new_direction)
            dy_m = individual_speed * self.dt * sin(new_direction)
            
            member['lat'] += dy_m / m_per_deg_lat
            member['lon'] += dx_m / m_per_deg_lon
            member['individual_direction'] = new_direction
            
            # Keep member in bounds
            member['lat'] = clamp(member['lat'], self.bounds.lat_min, self.bounds.lat_max)
            member['lon'] = clamp(member['lon'], self.bounds.lon_min, self.bounds.lon_max)
            
            return member['lat'], member['lon'], individual_speed
        
        return self.center_lat, self.center_lon, self.group_speed

class EnhancedRandomWaypoint:
    """Enhanced Random Waypoint with pause times and more realistic movement"""
    def _init_(self, bounds: Bounds, speed_mps_rng: Tuple[float, float], dt: float, 
                 pause_prob: float = 0.1, min_pause_s: float = 5.0, max_pause_s: float = 30.0):
        self.b = bounds; self.dt = dt
        self.vmin, self.vmax = speed_mps_rng
        self.pause_prob = pause_prob
        self.min_pause_s, self.max_pause_s = min_pause_s, max_pause_s
        
        self.lat = np.random.uniform(self.b.lat_min, self.b.lat_max)
        self.lon = np.random.uniform(self.b.lon_min, self.b.lon_max)
        self.pause_remaining = 0.0
        self.pick_new_waypoint()

    def pick_new_waypoint(self):
        self.tlat = np.random.uniform(self.b.lat_min, self.b.lat_max)
        self.tlon = np.random.uniform(self.b.lon_min, self.b.lon_max)
        self.speed = np.random.uniform(self.vmin, self.vmax)

    def step(self):
        # Handle pause state
        if self.pause_remaining > 0:
            self.pause_remaining -= self.dt
            return self.lat, self.lon, 0.0  # speed = 0 during pause
        
        # Random pause decision
        if np.random.rand() < self.pause_prob * self.dt:
            self.pause_remaining = np.random.uniform(self.min_pause_s, self.max_pause_s)
            return self.lat, self.lon, 0.0

        # Normal movement
        m_per_deg_lat = 111000.0
        m_per_deg_lon = 111000.0 * cos(radians(self.lat))
        dlat_deg = self.tlat - self.lat
        dlon_deg = self.tlon - self.lon
        dx_m = dlon_deg * m_per_deg_lon
        dy_m = dlat_deg * m_per_deg_lat
        dist_m = sqrt(dx_m*dx_m + dy_m*dy_m)
        step_m = self.speed * self.dt
        
        if dist_m < 1.0:
            self.pick_new_waypoint()
        else:
            ratio = min(1.0, step_m / (dist_m + 1e-9))
            self.lat = clamp(self.lat + dlat_deg * ratio, self.b.lat_min, self.b.lat_max)
            self.lon = clamp(self.lon + dlon_deg * ratio, self.b.lon_min, self.b.lon_max)
        return self.lat, self.lon, self.speed

class VehicularHighway:
    """Realistic highway mobility with lanes and realistic constraints"""
    def _init_(self, bounds: Bounds, dt: float, num_lanes: int = 3, lane_width_m: float = 3.5):
        self.b = bounds; self.dt = dt
        self.num_lanes = num_lanes
        self.lane_width_m = lane_width_m
        
        # Start in random lane
        self.lane = np.random.randint(0, num_lanes)
        self.lat = np.random.uniform(self.b.lat_min, self.b.lat_max)
        self.lon = np.random.uniform(self.b.lon_min, self.b.lon_max)
        
        # Highway-appropriate speed (80-120 km/h)
        self.speed = np.random.uniform(22, 33)  # m/s
        self.direction = np.random.choice([-1, 1])  # Forward or backward along highway

    def step(self):
        # Lane changing probability (low)
        if np.random.rand() < 0.01:  # 1% chance per step
            new_lane = self.lane + np.random.choice([-1, 1])
            self.lane = clamp(new_lane, 0, self.num_lanes - 1)
        
        # Speed variation (cruise control with small variations)
        self.speed += np.random.normal(0, 0.5)
        self.speed = clamp(self.speed, 15, 40)  # 54-144 km/h range
        
        # Movement along highway (simplified as lat movement)
        m_per_deg_lat = 111000.0
        dlat = (self.speed * self.dt * self.direction) / m_per_deg_lat
        self.lat = clamp(self.lat + dlat, self.b.lat_min, self.b.lat_max)
        
        # Slight lon offset based on lane
        lane_offset_m = (self.lane - self.num_lanes/2) * self.lane_width_m
        m_per_deg_lon = 111000.0 * cos(radians(self.lat))
        self.lon = self.b.lon_min + (self.b.lon_max - self.b.lon_min)/2 + lane_offset_m/m_per_deg_lon
        
        return self.lat, self.lon, self.speed

# ----------------------------
# Enhanced 3GPP Path Loss Models (TR 38.901 compliant)
# ----------------------------
def pathloss_3gpp_umi_los(f_hz, d_2d, h_bs=25, h_ut=1.5):
    """3GPP TR 38.901 UMi LOS path loss"""
    fc_ghz = f_hz / 1e9
    d_bp = 4 * h_bs * h_ut * fc_ghz * 1e9 / C
    
    if d_2d <= d_bp:
        return 32.4 + 21 * np.log10(d_2d) + 20 * np.log10(fc_ghz)
    else:
        return 32.4 + 40 * np.log10(d_2d) + 20 * np.log10(fc_ghz) - 9.5 * np.log10((d_bp)*2 + (h_bs - h_ut)*2)

def pathloss_3gpp_umi_nlos(f_hz, d_2d, h_bs=25, h_ut=1.5):
    """3GPP TR 38.901 UMi NLOS path loss"""
    fc_ghz = f_hz / 1e9
    pl_los = pathloss_3gpp_umi_los(f_hz, d_2d, h_bs, h_ut)
    return max(pl_los, 35.3 * np.log10(d_2d) + 22.4 + 21.3 * np.log10(fc_ghz) - 0.3 * (h_ut - 1.5))

def pathloss_3gpp_uma_los(f_hz, d_2d, h_bs=25, h_ut=1.5):
    """3GPP TR 38.901 UMa LOS path loss"""
    fc_ghz = f_hz / 1e9
    d_bp = 4 * h_bs * h_ut * fc_ghz * 1e9 / C
    
    if d_2d <= d_bp:
        return 28.0 + 22 * np.log10(d_2d) + 20 * np.log10(fc_ghz)
    else:
        return 28.0 + 40 * np.log10(d_2d) + 20 * np.log10(fc_ghz) - 9 * np.log10((d_bp)*2 + (h_bs - h_ut)*2)

def pathloss_3gpp_rma_los(f_hz, d_2d, h_bs=35, h_ut=1.5):
    """3GPP TR 38.901 RMa LOS path loss"""
    fc_ghz = f_hz / 1e9
    d_bp = 2 * pi * h_bs * h_ut * fc_ghz * 1e9 / C
    
    if d_2d <= d_bp:
        return 20 * np.log10(40 * pi * d_2d * fc_ghz / 3) + min(0.03 * h_bs*1.72, 10) * np.log10(d_2d) - min(0.044 * h_bs*1.72, 14.77) + 0.002 * np.log10(h_bs) * d_2d
    else:
        return 20 * np.log10(40 * pi * d_bp * fc_ghz / 3) + 40 * np.log10(d_2d / d_bp) + min(0.03 * h_bs*1.72, 10) * np.log10(d_2d) - min(0.044 * h_bs*1.72, 14.77) + 0.002 * np.log10(h_bs) * d_2d

# mmWave path loss (simplified)
def pathloss_mmwave(f_hz, d_m, blockage_prob=0.3):
    """Simplified mmWave path loss with blockage"""
    base_pl = 32.4 + 20 * np.log10(f_hz/1e9) + 20 * np.log10(d_m)
    
    # Additional loss due to atmospheric absorption (rough approximation)
    if f_hz > 20e9:
        atm_loss = 0.1 * d_m / 1000  # dB/km
        base_pl += atm_loss
    
    # Random blockage events
    if np.random.rand() < blockage_prob:
        base_pl += np.random.uniform(10, 30)  # Heavy blockage loss
    
    return base_pl

# ----------------------------
# Enhanced LOS Probability (3GPP TR 38.901)
# ----------------------------
def los_prob_3gpp_umi(d_2d):
    """3GPP TR 38.901 UMi LOS probability"""
    if d_2d <= 18:
        return 1.0
    else:
        return (18/d_2d) + np.exp(-d_2d/36) * (1 - 18/d_2d)

def los_prob_3gpp_uma(d_2d):
    """3GPP TR 38.901 UMa LOS probability"""
    if d_2d <= 18:
        return 1.0
    else:
        return (18/d_2d) + np.exp(-d_2d/63) * (1 - 18/d_2d)

def los_prob_3gpp_rma(d_2d):
    """3GPP TR 38.901 RMa LOS probability"""
    if d_2d <= 10:
        return 1.0
    else:
        return np.exp(-(d_2d - 10)/1000)

# ----------------------------
# Spatially Correlated Shadowing
# ----------------------------
class SpatialShadowing:
    """Spatially correlated lognormal shadowing"""
    def _init_(self, correlation_distance_m=50.0, std_db=8.0):
        self.corr_dist = correlation_distance_m
        self.std_db = std_db
        self.shadow_map = {}  # Cache for spatial correlation
        
    def get_shadowing(self, lat, lon):
        """Get correlated shadowing value for a position"""
        # Simple grid-based correlation
        grid_size = self.corr_dist / 111000.0  # Convert to degrees
        grid_lat = round(lat / grid_size) * grid_size
        grid_lon = round(lon / grid_size) * grid_size
        grid_key = (grid_lat, grid_lon)
        
        if grid_key not in self.shadow_map:
            self.shadow_map[grid_key] = np.random.normal(0, self.std_db)
            
        # Add some randomness for sub-grid variation
        base_shadow = self.shadow_map[grid_key]
        variation = np.random.normal(0, self.std_db * 0.3)  # 30% additional variation
        
        return base_shadow + variation

# ----------------------------
# Enhanced Jake's Fading Model
# ----------------------------
class JakesFading:
    """Jake's fading model using Bessel function autocorrelation"""
    def _init_(self, fd_hz, dt_s, num_samples_buffer=100):
        self.fd = fd_hz
        self.dt = dt_s
        self.buffer_size = num_samples_buffer
        self.buffer = np.random.randn(num_samples_buffer) + 1j * np.random.randn(num_samples_buffer)
        self.index = 0
        
    def next_sample(self):
        """Generate next correlated fading sample"""
        if self.index >= self.buffer_size - 1:
            # Regenerate buffer with correlation
            new_buffer = np.zeros(self.buffer_size, dtype=complex)
            for i in range(self.buffer_size):
                tau = i * self.dt
                rho = j0(2 * pi * self.fd * tau)  # Jake's autocorrelation
                if i == 0:
                    new_buffer[i] = np.random.randn() + 1j * np.random.randn()
                else:
                    noise = np.random.randn() + 1j * np.random.randn()
                    new_buffer[i] = rho * new_buffer[i-1] + np.sqrt(1 - rho**2) * noise
            self.buffer = new_buffer
            self.index = 0
            
        sample = self.buffer[self.index]
        self.index += 1
        return abs(sample)**2  # Return power

# ----------------------------
# Enhanced Network Features & Scheduling
# ----------------------------

# 2. More sophisticated interference - ADD HERE
class OFDMAScheduler:
    """OFDMA resource allocation and interference modeling"""
    def _init_(self, total_rbs: int = 50, users_per_bs: int = 20):
        self.total_rbs = total_rbs  # Resource blocks
        self.users_per_bs = users_per_bs
        self.allocation_history = {}
        
    def assign_resources(self, users_sinr: List[float], user_ids: List[int]) -> Dict[int, Dict]:
        """
        Assign RBs to users based on proportional fair scheduling
        Returns: {user_id: {'rbs': [list], 'power_per_rb': float}}
        """
        allocations = {}
        available_rbs = list(range(self.total_rbs))
        
        # Sort users by SINR (best channel conditions first)
        user_sinr_pairs = list(zip(user_ids, users_sinr))
        user_sinr_pairs.sort(key=lambda x: x[1], reverse=True)
        
        rbs_per_user = max(1, self.total_rbs // len(user_ids))
        
        for user_id, sinr_db in user_sinr_pairs:
            if len(available_rbs) == 0:
                break
                
            # Allocate RBs
            allocated_rbs = available_rbs[:rbs_per_user]
            available_rbs = available_rbs[rbs_per_user:]
            
            # Power allocation (water-filling approximation)
            power_per_rb = max(0.1, 1.0 - 0.1 * (20 - sinr_db))  # Simplified
            
            allocations[user_id] = {
                'rbs': allocated_rbs,
                'power_per_rb': power_per_rb,
                'allocated_rbs_count': len(allocated_rbs)
            }
            
        return allocations

    def calculate_interference(self, serving_bs_users: Dict, interfering_bs_users: Dict) -> Dict[int, float]:
        """Calculate inter-cell interference for OFDMA"""
        interference_map = {}
        
        for user_id, allocation in serving_bs_users.items():
            total_interference = 0.0
            user_rbs = set(allocation['rbs'])
            
            # Check interference from other BSs
            for int_user_id, int_allocation in interfering_bs_users.items():
                int_rbs = set(int_allocation['rbs'])
                
                # Interference only on overlapping RBs
                overlapping_rbs = user_rbs.intersection(int_rbs)
                if overlapping_rbs:
                    interference_power = (len(overlapping_rbs) * int_allocation['power_per_rb'] * 
                                        0.1)  # Distance-based attenuation factor
                    total_interference += interference_power
            
            interference_map[user_id] = max(1e-12, total_interference)
            
        return interference_map

class TrafficLoadModel:
    """Realistic traffic load patterns"""
    def _init_(self):
        self.hourly_load_profile = {
            # Hour: (mean_users_ratio, peak_data_demand_mbps)
            6: (0.2, 5),   # Early morning
            9: (0.8, 25),  # Morning peak
            12: (0.9, 30), # Lunch peak  
            15: (0.7, 20), # Afternoon
            18: (1.0, 35), # Evening peak
            21: (0.9, 40), # Prime time
            23: (0.4, 15), # Late evening
            2: (0.1, 2)    # Night
        }
    
    def get_current_load(self, timestamp: datetime) -> Tuple[float, float]:
        """Get current load based on time of day"""
        hour = timestamp.hour
        
        # Find closest hour in profile
        closest_hour = min(self.hourly_load_profile.keys(), 
                          key=lambda x: abs(x - hour))
        
        base_ratio, base_demand = self.hourly_load_profile[closest_hour]
        
        # Add some randomness
        load_ratio = base_ratio * np.random.uniform(0.8, 1.2)
        data_demand = base_demand * np.random.uniform(0.7, 1.3)
        
        return min(1.0, load_ratio), max(1.0, data_demand)
ENVIRONMENTS: Dict[str, Dict] = {
    "UMi":      {"type": "umi", "speed_kmh": (0, 60), "bounds": Bounds(28.45, 28.75, 76.95, 77.35), "shadowing_std": 4.0},
    "UMa":      {"type": "uma", "speed_kmh": (0, 80), "bounds": Bounds(13.00, 13.20, 80.15, 80.35), "shadowing_std": 6.0},
    "RMa":      {"type": "rma", "speed_kmh": (60, 130), "bounds": Bounds(19.00, 19.30, 72.75, 73.05), "shadowing_std": 4.0},
    "Highway":  {"type": "rma", "speed_kmh": (80, 120), "bounds": Bounds(17.20, 17.60, 78.30, 78.70), "shadowing_std": 3.0},
}

# Multiple base stations per environment for realistic coverage
BS_NETWORKS = {
    "UMi": [
        BaseStation(28.6139, 77.2090, 30, 25, 15, 0, 120, 0, 2.4e9),
        BaseStation(28.6200, 77.2150, 30, 25, 15, 120, 120, 1, 2.4e9),
        BaseStation(28.6180, 77.2030, 30, 25, 15, 240, 120, 2, 2.4e9),
    ],
    "UMa": [
        BaseStation(13.0827, 80.2707, 43, 35, 18, 0, 120, 0, 1.8e9),
        BaseStation(13.0900, 80.2800, 43, 35, 18, 120, 120, 1, 1.8e9),
    ],
    "RMa": [
        BaseStation(18.9878, 72.8365, 46, 45, 21, 0, 360, 0, 900e6),  # Omni
    ],
    "Highway": [
        BaseStation(17.3850, 78.4867, 46, 45, 21, 0, 360, 0, 900e6),  # Omni
    ],
}

# ----------------------------
# Enhanced Main Simulator
# ----------------------------
def simulate_enhanced_dataset(
    T_steps=600,
    dt_s=1.0,
    rx_per_env=50,
    mobility="enhanced_rwp",  # "enhanced_rwp", "vehicular", "gauss-markov"
    include_mmwave=True,
    use_spatial_shadowing=True,
    use_jakes_fading=True,
    include_handover=True,
    noise_dbm=-100.0,
    seed: Optional[int] = 12345
):
    if seed is not None:
        np.random.seed(seed)

    records = []
    start_time = datetime(2025, 1, 1, 9, 0, 0)

    # Initialize shadowing models per environment
    shadowing_models = {}
    if use_spatial_shadowing:
        for env, prof in ENVIRONMENTS.items():
            shadowing_models[env] = SpatialShadowing(
                correlation_distance_m=50.0,
                std_db=prof["shadowing_std"]
            )

    # Build RX agents with enhanced mobility
    rx_agents = []
    for env, prof in ENVIRONMENTS.items():
        for i in range(rx_per_env):
            b = prof["bounds"]
            speed_rng = tuple(v/3.6 for v in prof["speed_kmh"])
            
            # Enhanced mobility selection with new models
            if mobility == "enhanced_rwp":
                agent = EnhancedRandomWaypoint(bounds=b, speed_mps_rng=speed_rng, dt=dt_s)
            elif mobility == "vehicular" and env in ["Highway", "RMa"]:
                agent = VehicularHighway(bounds=b, dt=dt_s)
            elif mobility == "trace_driven":
                # Example: agent = TraceDrivenMobility("path/to/gps_trace.csv", dt_s)
                # Fallback to enhanced RWP if no trace file
                agent = EnhancedRandomWaypoint(bounds=b, speed_mps_rng=speed_rng, dt=dt_s)
            elif mobility == "group_mobility":
                # Create group mobility agent (returns one member of the group)
                agent = GroupMobility(bounds=b, dt=dt_s, group_size=5)
            else:
                # Fallback to Gauss-Markov
                from math import pi
                class GaussMarkov:
                    def _init_(self, bounds, dt, alpha=0.85, mean_speed=15.0, speed_var=5.0):
                        self.b = bounds; self.dt = dt; self.alpha = alpha
                        self.mean_speed = mean_speed; self.speed_var = speed_var
                        self.lat = np.random.uniform(self.b.lat_min, self.b.lat_max)
                        self.lon = np.random.uniform(self.b.lon_min, self.b.lon_max)
                        self.speed = abs(np.random.normal(mean_speed, speed_var))
                        self.theta = np.random.uniform(0, 2*pi)
                    
                    def step(self):
                        self.speed = self.alpha * self.speed + (1-self.alpha)*self.mean_speed + np.random.normal(0, self.speed_var)
                        self.theta = self.alpha * self.theta + (1-self.alpha)*np.random.uniform(0, 2*pi) + np.random.normal(0, 0.1)
                        m_per_deg_lat = 111000.0
                        m_per_deg_lon = 111000.0 * cos(radians(self.lat))
                        dx_m = self.speed * self.dt * cos(self.theta)
                        dy_m = self.speed * self.dt * sin(self.theta)
                        dlon_deg = dx_m / max(m_per_deg_lon, 1.0)
                        dlat_deg = dy_m / m_per_deg_lat
                        self.lat = clamp(self.lat + dlat_deg, self.b.lat_min, self.b.lat_max)
                        self.lon = clamp(self.lon + dlon_deg, self.b.lon_min, self.b.lon_max)
                        return self.lat, self.lon, max(self.speed, 0.0)
                
                agent = GaussMarkov(bounds=b, dt=dt_s, mean_speed=np.mean(speed_rng))

            rx_agents.append({
                "env": env,
                "agent": agent,
                "connected_bs_id": 0,  # For handover tracking
                "jakes_fading": None,  # Will be initialized per frequency
                "fading_buffer": {}    # Per frequency fading state
            })

    # Initialize enhanced network features
    ofdma_scheduler = OFDMAScheduler(total_rbs=50, users_per_bs=rx_per_env)
    traffic_model = TrafficLoadModel()
    
    # Frequency bands including mmWave
    freq_bands = [900e6, 1.8e9, 2.4e9, 3.5e9, 5e9]
    if include_mmwave:
        freq_bands.extend([28e9, 60e9])

    # Main simulation loop
    for t in range(T_steps):
        ts = start_time + timedelta(seconds=t*dt_s)
        
        # Get current traffic load
        load_ratio, data_demand_mbps = traffic_model.get_current_load(ts)
        
        for rx in rx_agents:
            env = rx["env"]
            agent = rx["agent"]
            lat, lon, speed = agent.step()
            
            # Get all base stations for this environment
            bs_list = BS_NETWORKS[env]
            
            # Find best serving BS (highest RSRP)
            best_bs = None
            best_rsrp = -float('inf')
            
            for bs in bs_list:
                d_m = haversine_m(lat, lon, bs.lat, bs.lon)
                angle_deg = calculate_angle_deg(lat, lon, bs.lat, bs.lon)
                
                # Select frequency (either fixed per BS or random)
                f_hz = bs.frequency_hz if hasattr(bs, 'frequency_hz') else float(np.random.choice(freq_bands))
                
                # Path loss calculation based on environment type and LOS
                env_type = ENVIRONMENTS[env]["type"]
                
                if env_type == "umi":
                    p_los = los_prob_3gpp_umi(d_m)
                    is_los = np.random.rand() < p_los
                    pl_db = pathloss_3gpp_umi_los(f_hz, d_m) if is_los else pathloss_3gpp_umi_nlos(f_hz, d_m)
                elif env_type == "uma":
                    p_los = los_prob_3gpp_uma(d_m)
                    is_los = np.random.rand() < p_los
                    pl_db = pathloss_3gpp_uma_los(f_hz, d_m) if is_los else pathloss_3gpp_uma_los(f_hz, d_m) + 10
                else:  # RMa
                    p_los = los_prob_3gpp_rma(d_m)
                    is_los = np.random.rand() < p_los
                    pl_db = pathloss_3gpp_rma_los(f_hz, d_m)
                
                # mmWave special handling
                if f_hz > 20e9:
                    pl_db = pathloss_mmwave(f_hz, d_m)
                
                # Antenna gain
                ant_gain_db = bs.antenna_gain_pattern(angle_deg)
                
                # Calculate RSRP for BS selection
                rsrp_dbm = bs.power_dbm + ant_gain_db - pl_db
                
                if rsrp_dbm > best_rsrp:
                    best_rsrp = rsrp_dbm
                    best_bs = bs
                    best_pl_db = pl_db
                    best_is_los = is_los
                    best_p_los = p_los
                    best_ant_gain = ant_gain_db
                    best_freq = f_hz
                    best_distance = d_m
                    best_angle = angle_deg
            
            # Handover detection
            handover_event = (rx["connected_bs_id"] != best_bs.sector_id) if include_handover else False
            rx["connected_bs_id"] = best_bs.sector_id
            
            # Enhanced shadowing
            if use_spatial_shadowing and env in shadowing_models:
                shadow_db = shadowing_models[env].get_shadowing(lat, lon)
            else:
                shadow_db = np.random.normal(0, ENVIRONMENTS[env]["shadowing_std"])
            
            # Enhanced fading with Jake's model
            if use_jakes_fading:
                fd = (speed / C) * best_freq  # Doppler frequency
                fading_key = f"{best_freq:.0f}"
                
                if fading_key not in rx["fading_buffer"]:
                    rx["fading_buffer"][fading_key] = JakesFading(fd, dt_s)
                
                # Update Doppler if speed changed significantly
                if abs(fd - rx["fading_buffer"][fading_key].fd) > 1.0:
                    rx["fading_buffer"][fading_key] = JakesFading(fd, dt_s)
                
                fading_power_lin = rx["fading_buffer"][fading_key].next_sample()
                fading_db = lin_to_db(fading_power_lin)
            else:
                # Simple Rayleigh fading
                fading_db = lin_to_db(np.random.exponential(1.0))
            
            # Total path loss and received power
            total_pl_db = best_pl_db + shadow_db - fading_db - best_ant_gain
            p_rx_dbm = best_bs.power_dbm - total_pl_db
            
            # Enhanced interference model
            interference_sources = [bs for bs in bs_list if bs != best_bs]
            total_interference_dbm = -120  # Background noise floor
            
            for interferer in interference_sources:
                int_d_m = haversine_m(lat, lon, interferer.lat, interferer.lon)
                int_angle = calculate_angle_deg(lat, lon, interferer.lat, interferer.lon)
                int_pl = best_pl_db * (int_d_m / best_distance) ** 2  # Simplified
                int_ant_gain = interferer.antenna_gain_pattern(int_angle)
                int_power_dbm = interferer.power_dbm + int_ant_gain - int_pl
                
                # Add to total interference
                total_interference_dbm = 10 * np.log10(
                    db_to_lin(total_interference_dbm) + db_to_lin(int_power_dbm)
                )
            
            # SINR calculation
            signal_lin = db_to_lin(p_rx_dbm)
            interference_lin = db_to_lin(total_interference_dbm)
            noise_lin = db_to_lin(noise_dbm)
            sinr_db = lin_to_db(signal_lin / (interference_lin + noise_lin))
            
            # Coverage and QoS classification
            coverage = 1 if sinr_db >= 0.0 else 0
            if sinr_db >= 15:
                qos_class = "Excellent"
            elif sinr_db >= 10:
                qos_class = "Good"
            elif sinr_db >= 5:
                qos_class = "Fair"
            elif sinr_db >= 0:
                qos_class = "Poor"
            else:
                qos_class = "Outage"
            
            # Throughput estimate (Shannon capacity)
            bandwidth_mhz = 20  # Typical LTE/5G bandwidth
            if best_freq > 20e9:  # mmWave
                bandwidth_mhz = 100
            
            capacity_mbps = bandwidth_mhz * np.log2(1 + db_to_lin(sinr_db))
            
            # Additional realistic features
            delay_spread_ns = np.random.exponential(50) if best_is_los else np.random.exponential(200)
            
            records.append({
                "Timestamp": ts.isoformat(),
                "Environment": env,
                "Environment_Type": ENVIRONMENTS[env]["type"].upper(),
                "Serving_BS_ID": best_bs.sector_id,
                "Handover_Event": handover_event,
                "Tx_Lat": float(best_bs.lat),
                "Tx_Lon": float(best_bs.lon),
                "Rx_Lat": float(lat),
                "Rx_Lon": float(lon),
                "Velocity_mps": float(speed),
                "Velocity_kmh": float(speed * 3.6),
                "Distance_m": float(best_distance),
                "AoA_deg": float(best_angle),
                "Tx_Freq_Hz": float(best_freq),
                "Freq_Band": "mmWave" if best_freq > 20e9 else "sub6GHz",
                "Tx_Power_dBm": float(best_bs.power_dbm),
                "Antenna_Gain_dBi": float(best_ant_gain),
                "Model_PL_dB": float(best_pl_db),
                "Shadowing_dB": float(shadow_db),
                "Fading_dB": float(fading_db),
                "Total_PL_dB": float(total_pl_db),
                "Rx_Power_dBm": float(p_rx_dbm),
                "Interference_dBm": float(total_interference_dbm),
                "SINR_dB": float(sinr_db),
                "Coverage": coverage,
                "QoS_Class": qos_class,
                "Capacity_Mbps": float(capacity_mbps),
                "LOS_Probability": float(best_p_los),
                "Is_LOS": best_is_los,
                "Delay_Spread_ns": float(delay_spread_ns),
                "Tx_Antenna_Height_m": float(best_bs.height_m),
                "Rx_Antenna_Height_m": 1.5,
                "Doppler_Hz": float((speed / C) * best_freq),
                "Bandwidth_MHz": bandwidth_mhz,
                "BS_Azimuth_deg": float(best_bs.azimuth_deg),
                "BS_Beamwidth_deg": float(best_bs.beamwidth_deg)
            })

    df = pd.DataFrame.from_records(records)
    
    # Add derived features for ML
    df["log10_Distance"] = np.log10(np.clip(df["Distance_m"], 1.0, None))
    df["Freq_GHz"] = df["Tx_Freq_Hz"] / 1e9
    df["Path_Loss_Efficiency"] = df["Model_PL_dB"] / (20 * np.log10(df["Distance_m"]) + 20 * np.log10(df["Freq_GHz"]))
    # Clean and validate velocity data
    df["Velocity_kmh"] = df["Velocity_kmh"].fillna(0)           # Step 1: Handle NaN velocities
    df["Velocity_kmh"] = np.maximum(df["Velocity_kmh"], 0)       # Step 2: No negative velocities

    # Fixed and robust categorical binning
    df["Mobility_Class"] = pd.cut(
        df["Velocity_kmh"],
        bins=[0, 5, 30, 80, 150],
        labels=["Stationary", "Pedestrian", "Vehicular", "High_Speed"],
        include_lowest=True,    # Step 3: Include 0 in 'Stationary'
        right=False             # Step 4: Left-closed bins: [0,5), [5,30), etc.
    )

    # Optional: Handle any remaining undefined/missing classes (should be very rare)
    df["Mobility_Class"] = df["Mobility_Class"].cat.add_categories("Unknown")
    df["Mobility_Class"] = df["Mobility_Class"].fillna("Unknown")

    
    return df

# ----------------------------
# Validation & Performance Analysis
# ----------------------------

# 3. Real data validation - ADD HERE  
def validate_with_drive_test_data(sim_df: pd.DataFrame, real_measurements_file: str = None) -> Dict:
    """
    Compare simulation results with real drive test data
    real_measurements_file: CSV with columns ['sinr_db', 'rsrp_dbm', 'distance_m', 'environment']
    """
    validation_results = {}
    
    if real_measurements_file and os.path.exists(real_measurements_file):
        real_df = pd.read_csv(real_measurements_file)
        
        print("📊 Validation Against Real Drive Test Data")
        print("=" * 50)
        
        # Compare SINR distributions
        for env in sim_df['Environment'].unique():
            if env in real_df['environment'].values:
                sim_sinr = sim_df[sim_df['Environment'] == env]['SINR_dB']
                real_sinr = real_df[real_df['environment'] == env]['sinr_db']
                
                # Statistical comparison
                from scipy import stats
                ks_stat, p_value = stats.ks_2samp(sim_sinr, real_sinr)
                
                validation_results[env] = {
                    'ks_statistic': ks_stat,
                    'p_value': p_value,
                    'sim_mean_sinr': sim_sinr.mean(),
                    'real_mean_sinr': real_sinr.mean(),
                    'sinr_difference': abs(sim_sinr.mean() - real_sinr.mean()),
                    'validation_status': 'PASS' if p_value > 0.05 else 'FAIL'
                }
                
                print(f"{env}:")
                print(f"  Sim SINR: {sim_sinr.mean():.1f} ± {sim_sinr.std():.1f} dB")
                print(f"  Real SINR: {real_sinr.mean():.1f} ± {real_sinr.std():.1f} dB") 
                print(f"  KS Test: p={p_value:.3f} ({'PASS' if p_value > 0.05 else 'FAIL'})")
                print()
    else:
        print("⚠  No real measurement data provided for validation")
        
        # Self-validation against expected 3GPP values
        print("📊 3GPP Standard Compliance Check")
        print("=" * 50)
        
        for env in sim_df['Environment'].unique():
            env_data = sim_df[sim_df['Environment'] == env]
            
            # Coverage rate should be reasonable
            coverage_rate = env_data['Coverage'].mean()
            expected_coverage = {'UMi': 0.85, 'UMa': 0.80, 'RMa': 0.90, 'Highway': 0.95}
            
            env_type = env_data['Environment_Type'].iloc[0] if 'Environment_Type' in env_data.columns else env
            expected = expected_coverage.get(env_type, 0.85)
            
            coverage_status = 'PASS' if abs(coverage_rate - expected) < 0.15 else 'FAIL'
            
            validation_results[env] = {
                'coverage_rate': coverage_rate,
                'expected_coverage': expected,
                'coverage_deviation': abs(coverage_rate - expected),
                'validation_status': coverage_status
            }
            
            print(f"{env} ({env_type}):")
            print(f"  Coverage: {coverage_rate:.1%} (expected ~{expected:.1%}) {coverage_status}")
    
    return validation_results

def generate_performance_report(df: pd.DataFrame, output_dir: str = "reports"):
    """Generate comprehensive performance analysis report"""
    os.makedirs(output_dir, exist_ok=True)
    
    report = []
    report.append("# Wireless Network Simulation Performance Report")
    report.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    report.append(f"Dataset Size: {len(df):,} records")
    report.append("")
    
    # Coverage Analysis
    report.append("## Coverage Performance")
    coverage_by_env = df.groupby('Environment')['Coverage'].agg(['mean', 'count'])
    for env, row in coverage_by_env.iterrows():
        report.append(f"- *{env}*: {row['mean']:.1%} coverage ({row['count']:,} samples)")
    report.append("")
    
    # SINR Distribution
    report.append("## SINR Distribution")
    sinr_stats = df.groupby('Environment')['SINR_dB'].agg(['mean', 'std', 'min', 'max'])
    for env, row in sinr_stats.iterrows():
        report.append(f"- *{env}*: {row['mean']:.1f} ± {row['std']:.1f} dB (range: {row['min']:.1f} to {row['max']:.1f})")
    report.append("")
    
    # Handover Analysis
    if 'Handover_Event' in df.columns:
        report.append("## Handover Performance") 
        handover_stats = df.groupby('Environment')['Handover_Event'].agg(['sum', 'mean'])
        for env, row in handover_stats.iterrows():
            ho_rate = row['mean'] * 100  # Convert to percentage
            report.append(f"- *{env}*: {row['sum']} handovers ({ho_rate:.2f}% of samples)")
        report.append("")
    
    # Frequency Band Analysis
    if 'Freq_Band' in df.columns:
        report.append("## Frequency Band Performance")
        freq_performance = df.groupby(['Environment', 'Freq_Band'])['SINR_dB'].mean().unstack()
        
        # Create table manually (without tabulate dependency)
        report.append("| Environment | " + " | ".join(freq_performance.columns) + " |")
        report.append("|" + "---|" * (len(freq_performance.columns) + 1))
        
        for env, row in freq_performance.iterrows():
            values = [f"{v:.1f}" if not pd.isna(v) else "N/A" for v in row.values]
            report.append(f"| {env} | " + " | ".join(values) + " |")
        report.append("")
    
    # Save report
    report_file = os.path.join(output_dir, "simulation_performance_report.md")
    with open(report_file, 'w') as f:
        f.write('\n'.join(report))
    
    print(f"📄 Performance report saved to: {report_file}")
    return report_file
def analyze_coverage_statistics(df):
    """Analyze coverage statistics by environment and frequency band"""
    print("📊 Coverage Analysis")
    print("=" * 50)
    
    coverage_by_env = df.groupby(['Environment', 'Freq_Band'])['Coverage'].agg(['mean', 'count']).round(3)
    print("Coverage Rate by Environment and Frequency Band:")
    print(coverage_by_env)
    print()
    
    sinr_stats = df.groupby('Environment')['SINR_dB'].agg(['mean', 'std', 'min', 'max']).round(2)
    print("SINR Statistics by Environment:")
    print(sinr_stats)
    print()
    
    qos_distribution = df.groupby(['Environment', 'QoS_Class']).size().unstack(fill_value=0)
    qos_percentage = qos_distribution.div(qos_distribution.sum(axis=1), axis=0) * 100
    print("QoS Class Distribution (%):")
    print(qos_percentage.round(1))

def analyze_handover_performance(df):
    """Analyze handover statistics"""
    if 'Handover_Event' in df.columns:
        handover_rate = df.groupby('Environment')['Handover_Event'].agg(['sum', 'count', 'mean']).round(3)
        handover_rate['Handovers_per_minute'] = handover_rate['sum'] / (handover_rate['count'] / 60)
        print("\n📱 Handover Analysis")
        print("=" * 50)
        print(handover_rate)

def validate_against_3gpp(df):
    """Basic validation against 3GPP expected values"""
    print("\n✅ 3GPP Compliance Check")
    print("=" * 50)
    
    # Check if path loss values are reasonable
    for env_type in df['Environment_Type'].unique():
        env_data = df[df['Environment_Type'] == env_type]
        avg_pl_100m = env_data[env_data['Distance_m'].between(90, 110)]['Model_PL_dB'].mean()
        
        # Expected path loss at 100m for different scenarios (approximate)
        expected_pl = {"UMI": 70, "UMA": 75, "RMA": 65}
        
        if env_type in expected_pl:
            deviation = abs(avg_pl_100m - expected_pl[env_type])
            status = "✅ PASS" if deviation < 10 else "❌ FAIL"
            print(f"{env_type}: Avg PL@100m = {avg_pl_100m:.1f} dB (expected ~{expected_pl[env_type]} dB) {status}")

# ----------------------------
# Demo run with enhanced features
# ----------------------------
if _name_ == "_main_":
    print("🚀 Starting Enhanced Wireless Network Simulation...")
    
    df = simulate_enhanced_dataset(
        T_steps=300,                    # 5 minutes simulation
        dt_s=1.0,
        rx_per_env=25,                  # 25 users per environment 
        mobility="enhanced_rwp",        # Enhanced mobility with pauses
        include_mmwave=True,            # Include mmWave bands
        use_spatial_shadowing=True,     # Spatially correlated shadowing
        use_jakes_fading=True,          # Jake's fading model
        include_handover=True,          # Multi-BS handover
        noise_dbm=-100.0,
        seed=2025
    )

    # Save enhanced dataset with error handling
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_file = rf"D:\VS code\Enhanced_Wireless_Dataset_Realistic_{timestamp}.csv"
    
    try:
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        df.to_csv(output_file, index=False)
        print(f"✅ Enhanced dataset saved to: {output_file}")
    except PermissionError:
        # Try alternative locations if D:\ has permission issues
        alternative_paths = [
            f"Enhanced_Wireless_Dataset_Realistic_{timestamp}.csv",  # Current directory
            rf"C:\Users\{os.getenv('USERNAME')}\Documents\Enhanced_Wireless_Dataset_Realistic_{timestamp}.csv",  # Documents
            rf"C:\temp\Enhanced_Wireless_Dataset_Realistic_{timestamp}.csv"  # Temp folder
        ]
        
        saved = False
        for alt_path in alternative_paths:
            try:
                # Create directory if it doesn't exist
                if os.path.dirname(alt_path):
                    os.makedirs(os.path.dirname(alt_path), exist_ok=True)
                df.to_csv(alt_path, index=False)
                output_file = alt_path
                print(f"✅ Enhanced dataset saved to: {output_file}")
                saved = True
                break
            except (PermissionError, OSError):
                continue
        
        if not saved:
            print("❌ Could not save to any location. Trying with basic filename...")
            try:
                basic_filename = f"wireless_dataset_{timestamp}.csv"
                df.to_csv(basic_filename, index=False)
                output_file = basic_filename
                print(f"✅ Enhanced dataset saved to: {os.path.abspath(output_file)}")
            except Exception as e:
                print(f"❌ Final save attempt failed: {e}")
                print("💡 Try closing any Excel files and run again, or check folder permissions")
                output_file = None

    print(f"✅ Enhanced dataset saved to: {output_file}")
    print(f"📊 Total records: {len(df):,}")
    print(f"🌐 Environments: {df['Environment'].nunique()}")
    print(f"📡 Base stations: {df['Serving_BS_ID'].nunique()}")
    print(f"📶 Frequency bands: {df['Freq_Band'].nunique()}")
    print()
    
    # Performance analysis
    analyze_coverage_statistics(df)
    analyze_handover_performance(df)
    validate_against_3gpp(df)
    
    # Display sample data
    print("\n📋 Sample Data Preview:")
    print("=" * 50)
    sample_cols = ['Environment', 'Velocity_kmh', 'Distance_m', 'Freq_Band', 
                   'SINR_dB', 'QoS_Class', 'Handover_Event', 'Capacity_Mbps']
    print(df[sample_cols].head(10).to_string(index=False))
    
    print(f"🎯 Key Improvements Implemented:")
    print("  • Multi-BS networks with realistic handovers")
    print("  • 3GPP TR 38.901 compliant path loss models")
    print("  • mmWave frequency support (28/60 GHz)")
    print("  • Spatially correlated shadowing")
    print("  • Jake's fading model for temporal correlation")
    print("  • Enhanced mobility (pauses, group, trace-driven)")
    print("  • OFDMA scheduling and realistic interference")
    print("  • Traffic load modeling with time-of-day patterns")
    print("  • QoS classification and capacity estimation")
    print("  • Directional antenna patterns")
    print("  • Comprehensive validation and reporting")
    
    # Generate performance report and validation
    report_file = generate_performance_report(df, output_dir=r"D:\VS code\reports")
    validation_results = validate_with_drive_test_data(df)
    
    print(f"\n✅ Simulation complete with enhanced realism!")
    print(f"📊 Validation results: {len([v for v in validation_results.values() if v.get('validation_status') == 'PASS'])} environments passed validation")

In [2]:
!pip install -q pysr sympy filterpy juliacall xgboost scikit-learn

import numpy as np
import pandas as pd
from pysr import PySRRegressor
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import StratifiedShuffleSplit
from filterpy.kalman import KalmanFilter
from sklearn.neighbors import NearestNeighbors
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import logging
import warnings
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ===================================================================
# Section 1: Adaptive Kalman Filter and Distance Calculation
# ===================================================================
def adaptive_kalman_filter(series):
    kf = KalmanFilter(dim_x=2, dim_z=1)
    kf.x = np.array([series.iloc[0], 0.0])
    kf.F = np.array([[1.0, 1.0], [0.0, 1.0]])
    kf.H = np.array([[1.0, 0.0]])
    kf.P *= 10.0
    kf.Q = np.array([[0.01, 0.0], [0.0, 0.01]])
    window = 10
    filtered = []
    for i in range(len(series)):
        if i < window:
            kf.R = np.var(series.iloc[:window]) if np.var(series.iloc[:window]) > 0 else 0.1
        else:
            kf.R = np.var(series.iloc[i-window:i]) if np.var(series.iloc[i-window:i]) > 0 else 0.1
        kf.predict()
        kf.update(series.iloc[i])
        filtered.append(kf.x[0])
    return filtered

def kalman_filter_gps_adaptive(df, lat_col='Rx_Lat', lon_col='Rx_Lon'):
    df[lat_col + '_kf'] = adaptive_kalman_filter(df[lat_col])
    df[lon_col + '_kf'] = adaptive_kalman_filter(df[lon_col])
    return df

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371000  # Earth radius in meters
    return c * r

# ===================================================================
# Section 2: Data Preparation Pipeline
# ===================================================================
def load_and_prepare_dataframe_v2(file_path):
    logging.info("📁 Loading dataset...")
    df = pd.read_csv(file_path)
    if 'Timestamp' in df.columns:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    logging.info("🔧 Applying adaptive Kalman filtering to GPS coordinates...")
    df = kalman_filter_gps_adaptive(df)
    logging.info("🗺️ Performing spatial clustering with adaptive eps tuning...")
    coords = np.radians(df[['Rx_Lat_kf', 'Rx_Lon_kf']].values)
    nbrs = NearestNeighbors(n_neighbors=10).fit(coords)
    dists, idxs = nbrs.kneighbors(coords)
    avg_dist = np.mean(dists[:, -1])
    eps_rad = avg_dist * 1.2
    logging.info(f"Using adaptive eps for DBSCAN clustering: {eps_rad:.6f} radians")
    clustering = DBSCAN(eps=eps_rad, min_samples=10, metric='haversine').fit(coords)
    df['markov_state'] = clustering.labels_
    logging.info("📏 Calculating distances...")
    if 'Distance_m' in df.columns:
        df['distance'] = df['Distance_m']
    else:
        df['distance'] = df.apply(lambda row: haversine(row['Tx_Lon'], row['Tx_Lat'], row['Rx_Lon_kf'], row['Rx_Lat_kf']), axis=1)
    logging.info("🏙️ Encoding environment types...")
    if 'Environment' in df.columns:
        env_map = {env: i+1 for i, env in enumerate(df['Environment'].unique())}
        df['environment_code'] = df['Environment'].map(env_map)
    elif 'Environment_Type' in df.columns:
        env_map = {env: i+1 for i, env in enumerate(df['Environment_Type'].unique())}
        df['environment_code'] = df['Environment_Type'].map(env_map)
    else:
        df['environment_code'] = 1
    return df

# ===================================================================
# Section 3: Markov Chain Feature Engineering
# ===================================================================
def estimate_higher_order_markov(states, order=2, smoothing=1e-3):
    sequences = [tuple(states[i:i + order]) for i in range(len(states) - order)]
    next_states = states[order:]
    unique_states = np.unique(states)
    state_to_idx = {s: i for i, s in enumerate(unique_states)}
    n = len(unique_states)
    counts, context_counts = {}, {}
    for ctx, nxt in zip(sequences, next_states):
        counts.setdefault(ctx, np.zeros(n))
        context_counts.setdefault(ctx, 0)
        counts[ctx][state_to_idx[nxt]] += 1
        context_counts[ctx] += 1
    trans_probs = {ctx: (counts[ctx] + smoothing) / (context_counts[ctx] + smoothing * n) for ctx in counts}
    return trans_probs, state_to_idx, unique_states

def get_transition_feature(state_seq, trans_probs, state_to_idx, unique_states, order=2, smoothing=1e-3):
    features, n = [], len(unique_states)
    for i in range(len(state_seq)):
        if i < order:
            features.append(0)
            continue
        ctx = tuple(state_seq[i - order:i])
        nxt = state_seq[i]
        if ctx in trans_probs:
            features.append(trans_probs[ctx][state_to_idx[nxt]])
        else:
            features.append(smoothing / (smoothing * n))
    return np.array(features)

def estimate_markov_chain(states, smoothing=1e-3):
    valid_states = states[states != -1]
    unique_states = np.unique(valid_states)
    n = len(unique_states)
    state_to_idx = {s: i for i, s in enumerate(unique_states)}
    counts = np.zeros((n, n)) + smoothing
    for i in range(len(valid_states) - 1):
        counts[state_to_idx[valid_states[i]], state_to_idx[valid_states[i + 1]]] += 1
    trans_probs = counts / counts.sum(axis=1, keepdims=True)
    return trans_probs, state_to_idx, unique_states

def extract_markov_features(trans_probs):
    eigvals, eigvecs = np.linalg.eig(trans_probs.T)
    idx = np.argmin(np.abs(eigvals - 1))
    stat_dist = np.real(eigvecs[:, idx])
    stat_dist /= stat_dist.sum()
    entropy = -np.sum(trans_probs * np.log(trans_probs + 1e-12), axis=1)
    return stat_dist, entropy

def add_markov_features_v2(df):
    logging.info("🔗 Computing advanced Markov chain features...")
    trans_probs, state_to_idx, unique_states = estimate_markov_chain(df['markov_state'].values)
    stat_dist, entropy_vals = extract_markov_features(trans_probs)
    df['stat_prob'] = df['markov_state'].map(lambda s: stat_dist[state_to_idx.get(s, 0)] if s in state_to_idx else 0)
    df['entropy'] = df['markov_state'].map(lambda s: entropy_vals[state_to_idx.get(s, 0)] if s in state_to_idx else 0)
    order, smoothing = 2, 1e-3
    tprobs_ho, sti_ho, ustates_ho = estimate_higher_order_markov(df['markov_state'].values, order, smoothing)
    df['trans_prob'] = get_transition_feature(df['markov_state'].values, tprobs_ho, sti_ho, ustates_ho, order, smoothing)
    return df

# ===================================================================
# Section 4: PySR Symbolic Regression Model
# ===================================================================
class ImprovedPySRPathLossModel:
    def __init__(self, niterations=40, populations=15, population_size=30, timeout_minutes=15):
        logging.info("🤖 Initializing Improved PySR symbolic regression model...")
        self.model = PySRRegressor(
            niterations=niterations,
            populations=populations,
            population_size=population_size,
            binary_operators=["+", "-", "*", "/"],
            unary_operators=["log", "sqrt", "abs", "exp", "sin"],
            parsimony=0.005,
            multithreading=True,
            procs=0,
            timeout_in_seconds=timeout_minutes * 60,
            model_selection="best",
            random_state=42,
            progress=True,
            verbosity=1
        )
    def fit(self, X, y, feature_names=None):
        logging.info("\n🚀 Starting SYMBOLIC REGRESSION with PySR!")
        logging.info(f"📊 Training dataset: {len(X)} samples × {X.shape[1]} features")
        logging.info(f"⏱️ Max runtime: {self.model.timeout_in_seconds // 60} minutes")
        self.model.fit(X, y, variable_names=feature_names)
        return self
    def predict(self, X):
        return self.model.predict(X)
    def get_best_equation(self):
        if hasattr(self.model, 'sympy') and self.model.sympy() is not None:
            return str(self.model.sympy())
        elif hasattr(self.model, 'equations_') and len(self.model.equations_) > 0:
            best_eq = self.model.equations_.iloc[-1]
            return str(best_eq['equation'])
        else:
            return "No equation discovered"

# ===================================================================
# Section 5: Ablation Study with PySR Model
# ===================================================================
def ablation_study_pysr(df, train_idx, val_idx, base_feature_cols, markov_feature_cols, target_col='Total_PL_dB'):
    logging.info("🧪 Starting ablation study with PySR model...")
    results = {}
    variants = {
        'Full Model': {'use_kf': True, 'use_markov': True},
        'Without Kalman Filter': {'use_kf': False, 'use_markov': True},
        'Without Markov Features': {'use_kf': True, 'use_markov': False},
        'Without Kalman & Markov': {'use_kf': False, 'use_markov': False},
    }
    for name, config in variants.items():
        logging.info(f"→ Evaluating variant: {name}")
        df_variant = df.copy()

        if not config['use_kf']:
            df_variant['Rx_Lat_kf'] = df_variant['Rx_Lat']
            df_variant['Rx_Lon_kf'] = df_variant['Rx_Lon']

        df_variant['distance'] = df_variant.apply(
            lambda row: haversine(row['Tx_Lon'], row['Tx_Lat'], row['Rx_Lon_kf'], row['Rx_Lat_kf']), axis=1)

        feature_cols = base_feature_cols.copy()
        if config['use_markov']:
            feature_cols += markov_feature_cols
        else:
            for mf in markov_feature_cols:
                if mf in df_variant.columns:
                    df_variant[mf] = 0.0

        train_df = df_variant.iloc[train_idx].reset_index(drop=True)
        val_df = df_variant.iloc[val_idx].reset_index(drop=True)

        X_train = train_df[feature_cols].values
        y_train = train_df[target_col].values
        X_val = val_df[feature_cols].values
        y_val = val_df[target_col].values

        pysr_model = ImprovedPySRPathLossModel(niterations=20, populations=8, population_size=20, timeout_minutes=10)
        pysr_model.fit(X_train, y_train, feature_names=feature_cols)

        val_pred = pysr_model.predict(X_val)

        val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
        val_mae = mean_absolute_error(y_val, val_pred)
        logging.info(f"   RMSE: {val_rmse:.4f}, MAE: {val_mae:.4f}")

        results[name] = {'RMSE': val_rmse, 'MAE': val_mae}
    return results

# ===================================================================
# Section 6: Classical and ML Benchmark Models
# ===================================================================
def free_space_path_loss(distance, freq):
    distance = np.maximum(distance, 1.0)
    freq = np.maximum(freq, 1e6)
    return 20 * np.log10(distance) + 20 * np.log10(freq) - 147.55

def hata_model(distance_km, freq_mhz, env_code):
    a_hr = np.zeros_like(env_code, dtype=float)
    urban_mask = (env_code == 1)
    suburban_mask = (env_code != 1)
    a_hr[urban_mask] = 3.2 * (np.log10(11.75))**2 - 4.97
    if np.any(suburban_mask):
        a_hr[suburban_mask] = (1.1*np.log10(freq_mhz) - 0.7) - (1.56*np.log10(freq_mhz) - 0.8)
    return (69.55 + 26.16*np.log10(freq_mhz) - 13.82*np.log10(30) - a_hr + (44.9 - 6.55*np.log10(30)) * np.log10(distance_km))

def cost231_hata_model(distance_km, freq_mhz, env_code):
    h_mt=1.5
    a_hm=(1.1*np.log10(freq_mhz)-0.7)*h_mt - (1.56*np.log10(freq_mhz)-0.8)
    urban_loss=3
    base_loss=46.3 + 33.9*np.log10(freq_mhz) - 13.82*np.log10(30) - a_hm
    clutter_adj=np.where(env_code==1, urban_loss, 0)
    distance_term=(44.9 - 6.55*np.log10(30))*np.log10(distance_km)
    return base_loss + distance_term + clutter_adj

def okumura_hata_model(distance_km, freq_mhz, env_code):
    a_hr = np.zeros_like(env_code, dtype=float)
    urban_mask = (env_code == 1)
    suburban_mask = (env_code != 1)
    a_hr[urban_mask] = 3.2 * (np.log10(11.75))**2 - 4.97
    a_hr[suburban_mask] = (1.1*np.log10(freq_mhz) - 0.7) - (1.56*np.log10(freq_mhz) - 0.8)
    return 69.55 + 26.16*np.log10(freq_mhz) - 13.82*np.log10(30) - a_hr + (44.9 - 6.55*np.log10(30))*np.log10(distance_km)

def log_distance_path_loss(distance_km, path_loss_exp=3.5, ref_loss=40):
    distance_km = np.maximum(distance_km, 1e-6)
    return ref_loss + 10*path_loss_exp*np.log10(distance_km)

def dual_slope_path_loss(distance_km, freq_mhz, breakpoint_km=0.5, n1=2.0, n2=3.5, ref_loss=40):
    d = np.maximum(distance_km, 1e-6)
    return np.where(d<breakpoint_km,
                    ref_loss + 10*n1*np.log10(d),
                    ref_loss + 10*n1*np.log10(breakpoint_km) + 10*n2*np.log10(d/breakpoint_km))

def floating_intercept_model(distance_km, alpha=35, beta=3.5):
    return alpha + 10*beta*np.log10(np.maximum(distance_km, 1e-6))

def cost231_walfisch_ikegami_model(distance_km, freq_mhz, env_code):
    base = 100 + 20*np.log10(distance_km) + 20*np.log10(freq_mhz)
    penalty = np.where(env_code == 1, 15, 5)
    return base + penalty

def fit_predict_mlp(X_train, y_train, X_test):
    mlp = MLPRegressor(hidden_layer_sizes=(50,30), max_iter=300, random_state=42)
    mlp.fit(X_train, y_train)
    return mlp.predict(X_test)

def fit_predict_xgboost(X_train, y_train, X_test):
    model = xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
    model.fit(X_train, y_train)
    return model.predict(X_test)

def fit_predict_svr(X_train, y_train, X_test):
    svr = SVR(kernel='rbf', C=10, epsilon=0.1)
    svr.fit(X_train, y_train)
    return svr.predict(X_test)

def fit_predict_knn(X_train, y_train, X_test):
    knn = KNeighborsRegressor(n_neighbors=5)
    knn.fit(X_train, y_train)
    return knn.predict(X_test)

def fit_predict_gradient_boosting(X_train, y_train, X_test):
    gbm = GradientBoostingRegressor(n_estimators=100, random_state=42)
    gbm.fit(X_train, y_train)
    return gbm.predict(X_test)

def fit_predict_elastic_net(X_train, y_train, X_test):
    enet = ElasticNet(alpha=0.1)
    enet.fit(X_train, y_train)
    return enet.predict(X_test)

def evaluate_benchmarks_full(df, y_true):
    freq_hz = df['Tx_Freq_Hz'].iloc[0] if 'Tx_Freq_Hz' in df.columns else 2.4e9
    freq_mhz = freq_hz / 1e6
    distance_safe = np.maximum(df['distance'], 1e-6)
    distance_km_safe = distance_safe / 1000

    results = {}

    try:
        # Classical path loss models
        results['Free_Space_PL'] = {'MSE': mean_squared_error(y_true, free_space_path_loss(distance_safe, freq_hz)),
                                   'MAE': mean_absolute_error(y_true, free_space_path_loss(distance_safe, freq_hz))}
        results['Hata_Model'] = {'MSE': mean_squared_error(y_true, hata_model(distance_km_safe, freq_mhz, df['environment_code'])),
                                'MAE': mean_absolute_error(y_true, hata_model(distance_km_safe, freq_mhz, df['environment_code']))}
        results['COST231_Hata'] = {'MSE': mean_squared_error(y_true, cost231_hata_model(distance_km_safe, freq_mhz, df['environment_code'])),
                                   'MAE': mean_absolute_error(y_true, cost231_hata_model(distance_km_safe, freq_mhz, df['environment_code']))}
        results['Okumura_Hata'] = {'MSE': mean_squared_error(y_true, okumura_hata_model(distance_km_safe, freq_mhz, df['environment_code'])),
                                   'MAE': mean_absolute_error(y_true, okumura_hata_model(distance_km_safe, freq_mhz, df['environment_code']))}
        results['Log_Distance'] = {'MSE': mean_squared_error(y_true, log_distance_path_loss(distance_km_safe)),
                                  'MAE': mean_absolute_error(y_true, log_distance_path_loss(distance_km_safe))}
        results['Dual_Slope'] = {'MSE': mean_squared_error(y_true, dual_slope_path_loss(distance_km_safe, freq_mhz)),
                                'MAE': mean_absolute_error(y_true, dual_slope_path_loss(distance_km_safe, freq_mhz))}
        results['Floating_Intercept'] = {'MSE': mean_squared_error(y_true, floating_intercept_model(distance_km_safe)),
                                        'MAE': mean_absolute_error(y_true, floating_intercept_model(distance_km_safe))}
        results['COST231_Walfisch_Ikegami'] = {'MSE': mean_squared_error(y_true, cost231_walfisch_ikegami_model(distance_km_safe, freq_mhz, df['environment_code'])),
                                               'MAE': mean_absolute_error(y_true, cost231_walfisch_ikegami_model(distance_km_safe, freq_mhz, df['environment_code']))}

        # ML/DL features
        base_features = ['distance', 'environment_code']
        additional_features = ['stat_prob', 'entropy', 'trans_prob']
        features = base_features + [f for f in additional_features if f in df.columns]
        X = df[features].values

        rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        rf.fit(X, y_true)
        preds_rf = rf.predict(X)
        results['Random_Forest'] = {'MSE': mean_squared_error(y_true, preds_rf), 'MAE': mean_absolute_error(y_true, preds_rf)}

        preds_svr = fit_predict_svr(X, y_true, X)
        results['SVR'] = {'MSE': mean_squared_error(y_true, preds_svr), 'MAE': mean_absolute_error(y_true, preds_svr)}

        preds_knn = fit_predict_knn(X, y_true, X)
        results['kNN'] = {'MSE': mean_squared_error(y_true, preds_knn), 'MAE': mean_absolute_error(y_true, preds_knn)}

        preds_gbm = fit_predict_gradient_boosting(X, y_true, X)
        results['Gradient_Boosting'] = {'MSE': mean_squared_error(y_true, preds_gbm), 'MAE': mean_absolute_error(y_true, preds_gbm)}

        preds_enet = fit_predict_elastic_net(X, y_true, X)
        results['ElasticNet'] = {'MSE': mean_squared_error(y_true, preds_enet), 'MAE': mean_absolute_error(y_true, preds_enet)}

        preds_xgb = fit_predict_xgboost(X, y_true, X)
        results['XGBoost'] = {'MSE': mean_squared_error(y_true, preds_xgb), 'MAE': mean_absolute_error(y_true, preds_xgb)}

        preds_mlp = fit_predict_mlp(X, y_true, X)
        results['MLP_NN'] = {'MSE': mean_squared_error(y_true, preds_mlp), 'MAE': mean_absolute_error(y_true, preds_mlp)}

        linreg = LinearRegression()
        linreg.fit(df[['distance']], y_true)
        preds_linreg = linreg.predict(df[['distance']])
        results['Linear_Regression'] = {'MSE': mean_squared_error(y_true, preds_linreg),
                                       'MAE': mean_absolute_error(y_true, preds_linreg)}

        return results

    except Exception as e:
        logging.warning(f"Benchmark eval warning: {e}")
        rf = RandomForestRegressor(n_estimators=50, random_state=42)
        rf.fit(df[['distance', 'environment_code']], y_true)
        rf_preds = rf.predict(df[['distance', 'environment_code']])
        return {'Random_Forest_Basic': {'MSE': mean_squared_error(y_true, rf_preds), 'MAE': mean_absolute_error(y_true, rf_preds)}}

# ===================================================================
# Section 7: Main Pipeline Execution
# ===================================================================
def main():
    logging.info("\n🎯 STARTING PySR PATH LOSS MODELING PIPELINE")
    logging.info("="*60)
    dataset_path = "/kaggle/input/pathloss/Enhanced_Wireless_Dataset_Realistic_20250824_131709.csv"  # <- Change as needed

    try:
        df = load_and_prepare_dataframe_v2(dataset_path)
    except FileNotFoundError:
        logging.error("Dataset file not found! Please check path.")
        return None, None, None, None, None
    except Exception as e:
        logging.error(f"Loading error: {e}")
        return None, None, None, None, None

    logging.info(f"Dataset shape: {df.shape}")
    if 'Environment' in df.columns:
        logging.info(f"Env types: {df['Environment'].unique()}")

    df = add_markov_features_v2(df)

    splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    for train_idx, val_idx in splitter.split(df, df['environment_code']):
        train = df.iloc[train_idx].reset_index(drop=True)
        val = df.iloc[val_idx].reset_index(drop=True)

    y_train = train['Total_PL_dB']
    y_val = val['Total_PL_dB']

    logging.info(f"Train samples: {len(train):,} Val samples: {len(val):,}")

    feature_columns = ['distance', 'environment_code', 'Tx_Freq_Hz', 'stat_prob', 'entropy', 'trans_prob']
    available_features = [col for col in feature_columns if col in train.columns]

    if len(train) > 2000:
        logging.info("Subsampling training set to 2000 samples for speed...")
        sample_idx = np.random.choice(len(train), 2000, replace=False)
        train_sample = train.iloc[sample_idx].reset_index(drop=True)
        y_train_sample = y_train.iloc[sample_idx].reset_index(drop=True)
    else:
        train_sample, y_train_sample = train, y_train

    X_train = train_sample[available_features].values
    X_val = val[available_features].values

    pysr_model = ImprovedPySRPathLossModel(niterations=30, populations=12, population_size=25, timeout_minutes=12)
    pysr_model.fit(X_train, y_train_sample.values, feature_names=available_features)

    train_preds = pysr_model.predict(X_train)
    val_preds = pysr_model.predict(X_val)

    train_rmse = np.sqrt(mean_squared_error(y_train_sample, train_preds))
    val_rmse = np.sqrt(mean_squared_error(y_val, val_preds))
    train_mae = mean_absolute_error(y_train_sample, train_preds)
    val_mae = mean_absolute_error(y_val, val_preds)

    logging.info(f"PYSR TRAIN RMSE: {train_rmse:.4f}, VAL RMSE: {val_rmse:.4f}")
    logging.info(f"PYSR TRAIN MAE: {train_mae:.4f}, VAL MAE: {val_mae:.4f}")

    best_equation = pysr_model.get_best_equation()
    logging.info(f"Best discovered equation:\n{best_equation}")

    base_features = ['distance', 'environment_code', 'Tx_Freq_Hz']
    markov_features = ['stat_prob', 'entropy', 'trans_prob']
    ablation_results = ablation_study_pysr(df, train_idx, val_idx, base_features, markov_features)

    comparison_results = {'PySR_Discovered_Full': {'RMSE': val_rmse, 'MAE': val_mae}}
    comparison_results.update(ablation_results)

    benchmarks = evaluate_benchmarks_full(val, y_val)

    for name, metrics in benchmarks.items():
        comparison_results[name] = {'RMSE': np.sqrt(metrics['MSE']), 'MAE': metrics['MAE']}

    print(f"\n📊 FINAL COMPARISON TABLE:")
    print(f"{'Model':<30} {'RMSE (dB)':<12} {'MAE (dB)':<12}")
    print("-"*54)
    for model_name, metrics in sorted(comparison_results.items()):
        print(f"{model_name:<30} {metrics['RMSE']:<12.4f} {metrics['MAE']:<12.4f}")

    return pysr_model, best_equation, val_rmse, ablation_results, benchmarks

# ===================================================================
# Run Pipeline
# ===================================================================
if __name__ == "__main__":
    print("🌟 STARTING PATH LOSS MODELING PIPELINE WITH ABLATION AND BENCHMARK")
    model, equation, performance, ablation, benchmarks = main()
    if model is not None:
        print("\n✅ PIPELINE COMPLETED SUCCESSFULLY")
        print(f"Equation:\n{equation}\nValidation RMSE: {performance:.4f} dB")
    else:
        print("\n❌ PIPELINE FAILED")


🌟 STARTING PATH LOSS MODELING PIPELINE WITH ABLATION AND BENCHMARK


[ Info: Started!



Expressions evaluated per second: 2.350e+04
Progress: 123 / 360 total iterations (34.167%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.893e+03  0.000e+00  y = 175.57
3           1.849e+03  1.181e-02  y = environment_code - -173.05
4           4.898e+02  1.328e+00  y = sqrt(distance * 2.0923)
6           4.865e+02  3.312e-03  y = sqrt(distance + distance) - -1.6342
7           2.492e+02  6.689e-01  y = sqrt(distance) - (sqrt(environment_code) * -33.524)
8           2.293e+02  8.306e-02  y = sqrt(distance) - (abs(sqrt(environment_code)) * -36.32...
                                      9)
11          1.628e+02  1.143e-01  y = sqrt(sqrt(distance)) * (20.794 - abs(-2.8586 / sin(Tx_...
                                      Freq_Hz)))
13          1.605e+02  7.035e-03  y = sqrt(sqr

[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.893e+03  0.000e+00  y = 175.57
3           1.849e+03  1.181e-02  y = environment_code - -173.05
4           4.109e+02  1.504e+00  y = sqrt(distance) - -53.655
6           3.764e+02  4.385e-02  y = sqrt(distance) + (environment_code - -53.655)
7           2.492e+02  4.123e-01  y = sqrt(distance) - (sqrt(environment_code) * -33.524)
8           2.293e+02  8.306e-02  y = sqrt(distance) - (abs(sqrt(environment_code)) * -36.32...
                                      9)
9           2.293e+02  4.745e-05  y = (sqrt(distance) + 0.5153) - (sqrt(environment_code) * ...
                                      -36.014)
11          1.618e+02  1.744e-01  y = (21.004 - (2.9476 / abs(sin(Tx_Freq_Hz)))) * sqrt(sqrt...
                                      (distance))
13          1.551e+02  2.111e-02  y = (23.006 - (3.0374 / abs(sin(Tx_Freq_Hz)))) * (

...

  - outputs/20250830_051429_svU2Ry/hall_of_fame.csv

Expressions evaluated per second: 2.880e+03
Progress: 20 / 160 total iterations (12.500%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.848e+03  0.000e+00  y = 176.26
4           5.016e+02  4.346e-01  y = sqrt(distance * 2.0785)
6           3.975e+02  1.164e-01  y = (sqrt(distance) + 33.571) / 0.87339
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════════════════════════════════════════════
Press 'q' and then <enter> to stop execution early.


[ Info: Note: you are running with more than 10,000 datapoints. You should consider turning on batching (`options.batching`), and also if you need that many datapoints. Unless you have a large amount of noise (in which case you should smooth your dataset first), generally < 10,000 datapoints is enough to find a functional form.
[ Info: Started!



Expressions evaluated per second: 2.680e+03
Progress: 38 / 160 total iterations (23.750%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           1.848e+03  0.000e+00  y = 176.26
4           5.016e+02  4.346e-01  y = sqrt(distance * 2.0785)
6           3.975e+02  1.164e-01  y = (sqrt(distance) + 33.571) / 0.87339
7           3.821e+02  3.948e-02  y = sqrt(abs(distance - -3565.6)) * 1.3028
16          3.701e+02  3.528e-03  y = (((environment_code + sqrt(distance)) + log(exp(sqrt(s...
                                      qrt(distance)) + 25.569))) * 1.0955) - -1.7451
20          3.554e+02  1.013e-02  y = ((1.145 * (environment_code + (sqrt(distance) + log(ex...
                                      p(sqrt(sqrt(distance)) + 11.259))))) + environment_code) -...
                              

[ Info: Final population:
[ Info: Results saved to:



📊 FINAL COMPARISON TABLE:
Model                          RMSE (dB)    MAE (dB)    
------------------------------------------------------
COST231_Hata                   35.5892      30.1278     
COST231_Walfisch_Ikegami       43.7715      38.6843     
Dual_Slope                     98.9927      92.4909     
ElasticNet                     14.3517      11.0541     
Floating_Intercept             107.9373     102.0078    
Free_Space_PL                  68.1707      56.6002     
Full Model                     110.5504     85.4046     
Gradient_Boosting              6.9815       5.3699      
Hata_Model                     35.2444      29.3126     
Linear_Regression              19.2780      15.5322     
Log_Distance                   103.2249     97.0078     
MLP_NN                         26.7469      20.8522     
Okumura_Hata                   35.2444      29.3126     
PySR_Discovered_Full           12.7563      10.1205     
Random_Forest                  4.5354       3.3620      
SVR   

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.neural_network import MLPRegressor
import xgboost as xgb
import logging
import warnings
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    r = 6371000  # Earth radius in meters
    return c * r

def load_and_prepare_dataframe_simple(file_path):
    logging.info("📁 Loading dataset...")
    df = pd.read_csv(file_path)
    if 'Timestamp' in df.columns:
        df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    logging.info("📏 Calculating distances...")
    df['distance'] = df.apply(lambda row: haversine(row['Tx_Lon'], row['Tx_Lat'], row['Rx_Lon'], row['Rx_Lat']), axis=1)
    logging.info("🏙️ Encoding environment types...")
    if 'Environment' in df.columns:
        env_map = {env: i + 1 for i, env in enumerate(df['Environment'].unique())}
        df['environment_code'] = df['Environment'].map(env_map)
    elif 'Environment_Type' in df.columns:
        env_map = {env: i + 1 for i, env in enumerate(df['Environment_Type'].unique())}
        df['environment_code'] = df['Environment_Type'].map(env_map)
    else:
        df['environment_code'] = 1
    return df

def free_space_path_loss(distance, freq):
    distance = np.maximum(distance, 1.0)
    freq = np.maximum(freq, 1e6)
    return 20 * np.log10(distance) + 20 * np.log10(freq) - 147.55

def hata_model(distance_km, freq_mhz, env_code):
    a_hr = np.zeros_like(env_code, dtype=float)
    urban_mask = (env_code == 1)
    suburban_mask = (env_code != 1)
    a_hr[urban_mask] = 3.2 * (np.log10(11.75)) ** 2 - 4.97
    if np.any(suburban_mask):
        a_hr[suburban_mask] = (1.1 * np.log10(freq_mhz) - 0.7) - (1.56 * np.log10(freq_mhz) - 0.8)
    return (69.55 + 26.16 * np.log10(freq_mhz) - 13.82 * np.log10(30) - a_hr + (44.9 - 6.55 * np.log10(30)) * np.log10(distance_km))

def cost231_hata_model(distance_km, freq_mhz, env_code):
    h_mt = 1.5
    a_hm = (1.1 * np.log10(freq_mhz) - 0.7) * h_mt - (1.56 * np.log10(freq_mhz) - 0.8)
    urban_loss = 3
    base_loss = 46.3 + 33.9 * np.log10(freq_mhz) - 13.82 * np.log10(30) - a_hm
    clutter_adj = np.where(env_code == 1, urban_loss, 0)
    distance_term = (44.9 - 6.55 * np.log10(30)) * np.log10(distance_km)
    return base_loss + distance_term + clutter_adj

def okumura_hata_model(distance_km, freq_mhz, env_code):
    a_hr = np.zeros_like(env_code, dtype=float)
    urban_mask = (env_code == 1)
    suburban_mask = (env_code != 1)
    a_hr[urban_mask] = 3.2 * (np.log10(11.75)) ** 2 - 4.97
    a_hr[suburban_mask] = (1.1 * np.log10(freq_mhz) - 0.7) - (1.56 * np.log10(freq_mhz) - 0.8)
    return 69.55 + 26.16 * np.log10(freq_mhz) - 13.82 * np.log10(30) - a_hr + (44.9 - 6.55 * np.log10(30)) * np.log10(distance_km)

def cost231_walfisch_ikegami_model(distance_km, freq_mhz, env_code):
    # Simplified placeholder version for urban environments
    L0 = 46.3 + 33.9 * np.log10(freq_mhz) - 13.82 * np.log10(30)  # base loss
    Lrts = -16.9 - 10 * np.log10(30) + 10 * np.log10(30)  # rooftop diff correction approx 0 here
    Lmsd = 0.0  # median street orientation loss, approx zero
    distance_term = 44.9 - 6.55 * np.log10(30)
    path_loss = L0 + (distance_term * np.log10(distance_km)) + Lrts + Lmsd
    path_loss += np.where(env_code == 1, 3, 0)  # clutter adjustment for urban
    return path_loss

def dual_slope_model(distance_km, freq_mhz):
    d0 = 0.1  # crossover distance km
    pl0 = 20 * np.log10(freq_mhz) + 20 * np.log10(d0) - 27.55
    slope1 = 2
    slope2 = 4
    pl = np.where(distance_km <= d0,
                  pl0 + 10 * slope1 * np.log10(distance_km / d0),
                  pl0 + 10 * slope1 * np.log10(1) + 10 * slope2 * np.log10(distance_km / d0))
    return pl

def floating_intercept_model(y_true):
    mean_val = np.mean(y_true)
    return np.full_like(y_true, mean_val)

def log_distance_model(distance_km):
    # Example coefficients for demonstration, usually fit on training data
    intercept = 30
    slope = 35
    return intercept + slope * np.log10(distance_km)

def nash_sutcliffe_efficiency(y_true, y_pred):
    numerator = np.sum((y_true - y_pred) ** 2)
    denominator = np.sum((y_true - np.mean(y_true)) ** 2)
    return 1 - (numerator / denominator)

def evaluate_generated_formula(df):
    dist = np.maximum(df['distance'], 1e-6)
    freq = df['Tx_Freq_Hz'].values
    val = dist ** 0.25 * (21.004284 - 2.947551 / np.abs(np.sin(freq)))
    return val

def evaluate_benchmarks_all(df, y_true):
    freq_hz = df['Tx_Freq_Hz'].iloc[0] if 'Tx_Freq_Hz' in df.columns else 2.4e9
    freq_mhz = freq_hz / 1e6
    distance_safe = np.maximum(df['distance'], 1e-6)
    distance_km_safe = distance_safe / 1000

    def compute_metrics(y_t, y_p):
        return {
            'RMSE': np.sqrt(mean_squared_error(y_t, y_p)),
            'MAE': mean_absolute_error(y_t, y_p),
            'R2': r2_score(y_t, y_p),
            'Adjusted_R2': 1 - (1 - r2_score(y_t, y_p)) * (len(y_t) - 1) / (len(y_t) - df[['distance', 'environment_code', 'Tx_Freq_Hz']].shape[1] - 1),
            'MedianAE': median_absolute_error(y_t, y_p),
            'NSE': nash_sutcliffe_efficiency(y_t, y_p),
            'Bias': np.mean(y_p - y_t),
            'Std_Error': np.std(y_p - y_t),
        }

    results = {}
    try:
        # Classical path loss models
        results['Free_Space_PL'] = compute_metrics(y_true, free_space_path_loss(distance_safe, freq_hz))
        results['Hata_Model'] = compute_metrics(y_true, hata_model(distance_km_safe, freq_mhz, df['environment_code']))
        results['COST231_Hata'] = compute_metrics(y_true, cost231_hata_model(distance_km_safe, freq_mhz, df['environment_code']))
        results['Okumura_Hata'] = compute_metrics(y_true, okumura_hata_model(distance_km_safe, freq_mhz, df['environment_code']))
        results['COST231_Walfisch_Ikegami'] = compute_metrics(y_true, cost231_walfisch_ikegami_model(distance_km_safe, freq_mhz, df['environment_code']))
        results['Dual_Slope'] = compute_metrics(y_true, dual_slope_model(distance_km_safe, freq_mhz))
        results['Floating_Intercept'] = compute_metrics(y_true, floating_intercept_model(y_true))
        results['Log_Distance'] = compute_metrics(y_true, log_distance_model(distance_km_safe))

        # Machine learning models
        base_features = ['distance', 'environment_code', 'Tx_Freq_Hz']
        X = df[base_features].values

        rf = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
        rf.fit(X, y_true)
        results['Random_Forest'] = compute_metrics(y_true, rf.predict(X))

        svr = SVR(kernel='rbf', C=10, epsilon=0.1)
        svr.fit(X, y_true)
        results['SVR'] = compute_metrics(y_true, svr.predict(X))

        knn = KNeighborsRegressor(n_neighbors=5)
        knn.fit(X, y_true)
        results['kNN'] = compute_metrics(y_true, knn.predict(X))

        gbm = GradientBoostingRegressor(n_estimators=100, random_state=42)
        gbm.fit(X, y_true)
        results['Gradient_Boosting'] = compute_metrics(y_true, gbm.predict(X))

        enet = ElasticNet(alpha=0.1)
        enet.fit(X, y_true)
        results['ElasticNet'] = compute_metrics(y_true, enet.predict(X))

        mlp = MLPRegressor(hidden_layer_sizes=(50, 30), max_iter=300, random_state=42)
        mlp.fit(X, y_true)
        results['MLP_NN'] = compute_metrics(y_true, mlp.predict(X))

        linreg_full = LinearRegression()
        linreg_full.fit(df[base_features], y_true)
        results['Full_Model'] = compute_metrics(y_true, linreg_full.predict(df[base_features]))

        linreg_dist = LinearRegression()
        linreg_dist.fit(df[['distance']], y_true)
        results['Linear_Regression'] = compute_metrics(y_true, linreg_dist.predict(df[['distance']]))

        xgb_reg = xgb.XGBRegressor(n_estimators=100, random_state=42, n_jobs=-1, verbosity=0)
        xgb_reg.fit(X, y_true)
        results['XGBoost'] = compute_metrics(y_true, xgb_reg.predict(X))

        preds_pysr_formula = evaluate_generated_formula(df)
        results['PySR_Discovered_Full'] = compute_metrics(y_true, preds_pysr_formula)

        return results

    except Exception as e:
        logging.warning(f"Benchmark evaluation warning: {e}")
        rf_basic = RandomForestRegressor(n_estimators=50, random_state=42)
        rf_basic.fit(df[['distance', 'environment_code']], y_true)
        rf_preds = rf_basic.predict(df[['distance', 'environment_code']])
        return {'Random_Forest_Basic': compute_metrics(y_true, rf_preds)}

def main_benchmark_only():
    logging.info("\n🎯 STARTING PATH LOSS BENCHMARKING ONLY")
    dataset_path = "/kaggle/input/pathloss/Enhanced_Wireless_Dataset_Realistic_20250824_131709.csv"  # Adjust path as needed
    try:
        df = load_and_prepare_dataframe_simple(dataset_path)
    except FileNotFoundError:
        logging.error("Dataset file not found! Please check path.")
        return
    except Exception as e:
        logging.error(f"Loading error: {e}")
        return

    if 'Total_PL_dB' not in df.columns:
        logging.error("Target column 'Total_PL_dB' not found in dataset.")
        return

    y_true = df['Total_PL_dB'].values
    benchmark_results = evaluate_benchmarks_all(df, y_true)

    print("\n📊 FINAL BENCHMARKING RESULTS:")
    for model_name, metrics in sorted(benchmark_results.items()):
        print(f"\nModel: {model_name}")
        print(f"RMSE (dB): {metrics['RMSE']:.4f}")
        print(f"MAE (dB): {metrics['MAE']:.4f}")
        print(f"R-squared: {metrics['R2']:.4f}")
        print(f"Adjusted R-squared: {metrics['Adjusted_R2']:.4f}")
        print(f"Median Absolute Error: {metrics['MedianAE']:.4f}")
        print(f"Nash–Sutcliffe Efficiency: {metrics['NSE']:.4f}")
        print(f"Bias (Mean Error): {metrics['Bias']:.4f}")
        print(f"Standard Deviation of Error: {metrics['Std_Error']:.4f}")

if __name__ == "__main__":
    main_benchmark_only()



📊 FINAL BENCHMARKING RESULTS:

Model: COST231_Hata
RMSE (dB): 35.9374
MAE (dB): 31.0812
R-squared: 0.2997
Adjusted R-squared: 0.2997
Median Absolute Error: 28.5750
Nash–Sutcliffe Efficiency: 0.2997
Bias (Mean Error): 4.4349
Standard Deviation of Error: 35.6627

Model: COST231_Walfisch_Ikegami
RMSE (dB): 37.7605
MAE (dB): 29.2353
R-squared: 0.2269
Adjusted R-squared: 0.2268
Median Absolute Error: 24.5207
Nash–Sutcliffe Efficiency: 0.2269
Bias (Mean Error): -12.4108
Standard Deviation of Error: 35.6627

Model: Dual_Slope
RMSE (dB): 79.1642
MAE (dB): 71.3202
R-squared: -2.3980
Adjusted R-squared: -2.3983
Median Absolute Error: 61.5086
Nash–Sutcliffe Efficiency: -2.3980
Bias (Mean Error): -71.3202
Standard Deviation of Error: 34.3568

Model: ElasticNet
RMSE (dB): 13.9292
MAE (dB): 10.4619
R-squared: 0.8948
Adjusted R-squared: 0.8948
Median Absolute Error: 7.6256
Nash–Sutcliffe Efficiency: 0.8948
Bias (Mean Error): 0.0000
Standard Deviation of Error: 13.9292

Model: Floating_Intercept
RMSE